# GSM CalcJob Data Transfer Diagnostic Notebook

This notebook focuses specifically on diagnosing the "Parameters are required" error in the data transfer between AiiDA CalcJob and the GSM CLI.

## Problem Statement
The workchain fails with exit code 301 and the error message "Parameters are required", indicating that the simulation_request.json file is either:
1. Not being created properly
2. Not accessible to the GSM CLI
3. In the wrong format
4. Missing required data

## Systematic Approach
We will test each step of the data transfer pipeline systematically:
1. **Input Data Preparation** - Verify inputs are correct
2. **File Creation** - Test if simulation_request.json is created
3. **File Content** - Verify the JSON structure and content
4. **File Accessibility** - Test if the CLI can access the file
5. **CLI Interface** - Test the GSM CLI directly with the generated file
6. **End-to-End Test** - Full CalcJob execution with detailed monitoring

## 1. Initial Setup and Environment

In [ ]:
# Essential imports
import aiida
aiida.load_profile()

from aiida import orm
from aiida.engine import submit, run
from aiida.orm import Dict, Str, load_node
from aiida.common.folders import Folder
import json
import os
import tempfile
import subprocess
import time

print("✅ AiiDA loaded successfully")
print(f"Profile: {aiida.get_profile().name}")

In [ ]:
# Load our calculation and workchain classes
from bmcs_matmod.aiida_plugins.calculations import GSMSimulationCalculation
from bmcs_matmod.aiida_plugins.workchains import GSMMonotonicWorkChain

# Load the GSM code
gsm_code = orm.load_code('gsm-cli-dev-shell@localhost')
print(f"✅ GSM Code loaded: {gsm_code.label}")
print(f"   Executable: {gsm_code.filepath_executable}")

## 2. Input Data Preparation Test

First, let's verify that our input data preparation is correct and matches what the GSM CLI expects.

In [ ]:
# Test input data preparation
print("=== INPUT DATA PREPARATION TEST ===")

# Define test inputs that should work
test_inputs = {
    'code': gsm_code,
    'gsm_model': Str('GSM1D_ED'),  # Simple elasto-damage model
    'formulation': Str('total'),
    'material_parameters': Dict({
        'E': 30000.0,    # Young's modulus
        'S': 1.0,        # Damage parameter
        'c': 2.0,        # Damage parameter
        'r': 0.9,        # Damage parameter
        'eps_0': 0.001   # Threshold strain
    }),
    'loading_data': Dict({
        'type': 'monotonic',
        'max_strain': 0.01,
        'n_steps': 20
    })
}

print("📋 Test inputs defined:")
for key, value in test_inputs.items():
    if key != 'code':
        if hasattr(value, 'value'):
            print(f"  {key}: {value.value}")
        elif hasattr(value, 'get_dict'):
            print(f"  {key}: {value.get_dict()}")
        else:
            print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value.label}")

print("\n✅ Input data preparation completed")

## 3. File Creation Test

Test if the `prepare_for_submission` method creates the correct `simulation_request.json` file.

In [ ]:
# Test file creation logic
print("=== FILE CREATION TEST ===")

# Simulate the prepare_for_submission logic
with tempfile.TemporaryDirectory() as temp_dir:
    print(f"🔧 Using temporary directory: {temp_dir}")
    
    # Create folder object
    folder = Folder(temp_dir)
    
    # Extract the same logic as in prepare_for_submission
    from bmcs_matmod.gsm_lagrange.gsm_def_registry import get_gsm_defs
    
    # Get inputs
    model_key = test_inputs['gsm_model'].value
    all_material_params = test_inputs['material_parameters'].get_dict()
    all_loading_data = test_inputs['loading_data'].get_dict()
    
    print(f"📊 Processing model: {model_key}")
    print(f"📊 Material parameters: {all_material_params}")
    print(f"📊 Loading data: {all_loading_data}")
    
    # Get model configuration from registry
    try:
        gsm_catalog = get_gsm_defs(debug=False)
        if model_key in gsm_catalog:
            print(f"✅ Model '{model_key}' found in registry")
            model_parameters = all_material_params
        else:
            print(f"⚠️ Model '{model_key}' not in registry, using all parameters")
            model_parameters = all_material_params
    except Exception as e:
        print(f"❌ Registry error: {e}")
        model_parameters = all_material_params
    
    # Filter loading data
    valid_loading_fields = {
        'time', 'strain', 'stress', 'max_strain', 'min_strain', 'n_steps', 'type',
        'strain_rate', 'cycles', 'R_ratio', 'loading_type', 'loading_history'
    }
    filtered_loading_data = {k: v for k, v in all_loading_data.items() if k in valid_loading_fields}
    
    print(f"📊 Filtered loading data: {filtered_loading_data}")
    
    # Create simulation request
    simulation_request = {
        "model": model_key,
        "formulation": test_inputs['formulation'].value,
        "parameters": {"parameters": model_parameters},  # Note the nested structure
        "loading": filtered_loading_data
    }
    
    print(f"\n📄 Generated simulation request:")
    print(json.dumps(simulation_request, indent=2))
    
    # Write to file
    try:
        with folder.open('simulation_request.json', 'w') as f:
            json.dump(simulation_request, f, indent=2)
        print("\n✅ File written successfully")
        
        # Verify file exists and read it back
        file_path = os.path.join(temp_dir, 'simulation_request.json')
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                written_content = f.read()
            print("\n📄 File content verification:")
            print(written_content)
            
            # Parse back to verify JSON is valid
            try:
                parsed_data = json.loads(written_content)
                print("\n✅ JSON is valid and parseable")
                
                # Store for next test
                generated_json_content = written_content
                generated_json_path = file_path
                
            except json.JSONDecodeError as e:
                print(f"❌ JSON parse error: {e}")
        else:
            print("❌ File was not created")
            
    except Exception as e:
        print(f"❌ File creation error: {e}")
        import traceback
        traceback.print_exc()

## 4. CLI Interface Test

Test the GSM CLI directly with our generated JSON file to see if the format is correct.

In [ ]:
# Test GSM CLI directly with our generated file
print("=== CLI INTERFACE TEST ===")

# First, test if the CLI is accessible
cli_path = '/home/rch/Coding/bmcs_matmod/gsm_cli_shell.sh'
print(f"🔧 Testing CLI at: {cli_path}")

if os.path.exists(cli_path):
    print("✅ CLI shell script exists")
    
    # Test basic CLI functionality
    try:
        result = subprocess.run([cli_path, '--help'], 
                              capture_output=True, text=True, timeout=10)
        print(f"📊 CLI help test:")
        print(f"   Return code: {result.returncode}")
        print(f"   Stdout: {result.stdout[:200]}..." if len(result.stdout) > 200 else f"   Stdout: {result.stdout}")
        if result.stderr:
            print(f"   Stderr: {result.stderr[:200]}..." if len(result.stderr) > 200 else f"   Stderr: {result.stderr}")
    except subprocess.TimeoutExpired:
        print("⚠️ CLI help command timed out")
    except Exception as e:
        print(f"❌ CLI help test failed: {e}")
        
else:
    print("❌ CLI shell script not found")

# Test with our generated JSON file
if 'generated_json_content' in locals():
    print("\n🧪 Testing CLI with generated JSON...")
    
    # Create a test file in a temporary location
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
        f.write(generated_json_content)
        test_json_path = f.name
    
    try:
        # Test the CLI with our JSON file
        cmd = [cli_path, '--execute-request', test_json_path, '--json-output']
        print(f"🚀 Running command: {' '.join(cmd)}")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        
        print(f"\n📊 CLI execution results:")
        print(f"   Return code: {result.returncode}")
        print(f"   Stdout: {result.stdout}")
        if result.stderr:
            print(f"   Stderr: {result.stderr}")
            
        if result.returncode == 0:
            print("\n✅ CLI executed successfully with our JSON!")
            try:
                # Try to parse the output as JSON
                output_data = json.loads(result.stdout)
                print("✅ Output is valid JSON")
                if 'status' in output_data:
                    print(f"   Status: {output_data['status']}")
            except json.JSONDecodeError:
                print("⚠️ Output is not valid JSON")
        else:
            print("❌ CLI execution failed")
            if "Parameters are required" in result.stdout or "Parameters are required" in result.stderr:
                print("🎯 FOUND THE ISSUE: CLI reports 'Parameters are required'")
                print("   This means our JSON format is incorrect for the CLI")
                
    except subprocess.TimeoutExpired:
        print("⚠️ CLI command timed out")
    except Exception as e:
        print(f"❌ CLI test failed: {e}")
    finally:
        # Clean up
        if os.path.exists(test_json_path):
            os.unlink(test_json_path)
            
else:
    print("❌ No generated JSON content available for testing")

## 5. JSON Format Analysis

Let's analyze what the CLI expects vs what we're providing.

In [ ]:
# Analyze JSON format requirements
print("=== JSON FORMAT ANALYSIS ===")

# Let's examine the CLI source to understand the expected format
cli_module_path = '/home/rch/Coding/bmcs_matmod/bmcs_matmod/gsm_lagrange/cli_gsm.py'

if os.path.exists(cli_module_path):
    print(f"📖 Examining CLI source: {cli_module_path}")
    
    # Look for parameter parsing logic
    with open(cli_module_path, 'r') as f:
        cli_source = f.read()
    
    # Find lines related to parameter processing
    lines = cli_source.split('\n')
    param_lines = []
    for i, line in enumerate(lines):
        if any(keyword in line.lower() for keyword in ['parameter', 'param', 'json', 'request']):
            param_lines.append((i+1, line.strip()))
    
    print(f"\n📋 Found {len(param_lines)} relevant lines in CLI source:")
    for line_num, line in param_lines[:20]:  # Show first 20 matches
        print(f"   {line_num:4d}: {line}")
        
else:
    print("❌ CLI source not found")

# Let's also test different JSON structures to see what works
print("\n🧪 Testing different JSON structures...")

# Structure 1: Our current structure
structure1 = {
    "model": "GSM1D_ED",
    "formulation": "total", 
    "parameters": {"parameters": {
        "E": 30000.0,
        "S": 1.0
    }},
    "loading": {
        "type": "monotonic",
        "max_strain": 0.01,
        "n_steps": 20
    }
}

# Structure 2: Flattened parameters
structure2 = {
    "model": "GSM1D_ED",
    "formulation": "total",
    "parameters": {
        "E": 30000.0,
        "S": 1.0
    },
    "loading": {
        "type": "monotonic", 
        "max_strain": 0.01,
        "n_steps": 20
    }
}

# Structure 3: Alternative naming
structure3 = {
    "model": "GSM1D_ED",
    "formulation": "total",
    "material_parameters": {
        "E": 30000.0,
        "S": 1.0
    },
    "loading_data": {
        "type": "monotonic",
        "max_strain": 0.01,
        "n_steps": 20
    }
}

structures = [
    ("Current (nested parameters)", structure1),
    ("Flattened parameters", structure2), 
    ("Alternative naming", structure3)
]

print("\n📊 Comparing structures:")
for name, struct in structures:
    print(f"\n{name}:")
    print(json.dumps(struct, indent=2))

## 6. CalcJob Execution Test

Now let's test the actual CalcJob execution with detailed monitoring.

In [ ]:
# Test actual CalcJob execution
print("=== CALCJOB EXECUTION TEST ===")

print("🚀 Submitting CalcJob...")
calc_node = submit(GSMSimulationCalculation, **test_inputs)
print(f"   Submitted with PK: {calc_node.pk}")

# Store the PK for monitoring
current_calc_pk = calc_node.pk

In [ ]:
# Monitor the CalcJob execution
print("=== CALCJOB MONITORING ===")

if 'current_calc_pk' in locals():
    calc = load_node(current_calc_pk)
    print(f"Monitoring CalcJob PK: {calc.pk}")
    
    # Wait for completion
    for i in range(30):  # Wait up to 30 seconds
        state = calc.get_state()
        print(f"  Attempt {i+1}: State = {state}")
        
        if calc.is_finished:
            break
        time.sleep(1)
    
    # Analyze results
    print(f"\n📊 Final Status:")
    print(f"  State: {calc.get_state()}")
    print(f"  Finished: {calc.is_finished}")
    print(f"  Successful: {calc.is_finished_ok if calc.is_finished else 'N/A'}")
    print(f"  Exit code: {calc.exit_code if calc.is_finished else 'N/A'}")
    
    if calc.is_finished:
        if calc.is_finished_ok:
            print("\n🎉 SUCCESS!")
        else:
            print("\n❌ FAILED - Analyzing...")
            
            # Check remote working directory
            if 'remote_folder' in calc.outputs:
                remote_folder = calc.outputs.remote_folder
                remote_path = remote_folder.get_remote_path()
                print(f"\n📁 Remote working directory: {remote_path}")
                
                # List directory contents
                try:
                    result = subprocess.run(['ls', '-la', remote_path], 
                                           capture_output=True, text=True)
                    print("\n📋 Directory contents:")
                    print(result.stdout)
                    
                    # Check if simulation_request.json exists
                    json_path = os.path.join(remote_path, 'simulation_request.json')
                    if os.path.exists(json_path):
                        print("\n✅ simulation_request.json exists!")
                        with open(json_path, 'r') as f:
                            actual_content = f.read()
                        print("📄 Actual file content:")
                        print(actual_content)
                        
                        # Test CLI with this exact file
                        print("\n🧪 Testing CLI with actual file...")
                        cmd = [cli_path, '--execute-request', json_path, '--json-output']
                        result = subprocess.run(cmd, capture_output=True, text=True, cwd=remote_path)
                        print(f"CLI test result: {result.returncode}")
                        print(f"Stdout: {result.stdout}")
                        print(f"Stderr: {result.stderr}")
                        
                    else:
                        print("\n❌ simulation_request.json does NOT exist!")
                        print("   This confirms the file creation issue.")
                        
                except Exception as e:
                    print(f"Error accessing remote directory: {e}")
            
            # Check retrieved files
            if 'retrieved' in calc.outputs:
                retrieved = calc.outputs.retrieved
                file_list = retrieved.list_object_names()
                print(f"\n📁 Retrieved files: {file_list}")
                
                # Check error logs
                for log_file in ['gsm_simulation.err', 'gsm_simulation.out']:
                    if log_file in file_list:
                        content = retrieved.get_object_content(log_file)
                        print(f"\n📄 {log_file}:")
                        print(content)
                        
    else:
        print("\n⏳ Still running after 30 seconds")
        
else:
    print("❌ No current calculation to monitor")

## 7. Root Cause Analysis

Based on the test results above, let's identify the exact cause of the "Parameters are required" error.

In [ ]:
# Root cause analysis
print("=== ROOT CAUSE ANALYSIS ===")

print("\n🔍 Summary of findings:")
print("\nBased on the tests above, the 'Parameters are required' error can be caused by:")
print("\n1. 📄 FILE CREATION ISSUES:")
print("   - simulation_request.json not created at all")
print("   - File created but in wrong location")
print("   - File created but with wrong permissions")

print("\n2. 📊 JSON FORMAT ISSUES:")
print("   - Wrong JSON structure (nested vs flat parameters)")
print("   - Wrong field names (parameters vs material_parameters)")
print("   - Missing required fields")
print("   - Invalid JSON syntax")

print("\n3. 🔧 CLI INTERFACE ISSUES:")
print("   - CLI not finding the file")
print("   - CLI expecting different file name")
print("   - Working directory mismatch")
print("   - Command line arguments incorrect")

print("\n4. 🔄 EXECUTION FLOW ISSUES:")
print("   - prepare_for_submission not called")
print("   - Exception during file creation")
print("   - AiiDA caching old code")

print("\n🎯 NEXT STEPS:")
print("Based on which test failed above, focus on:")
print("\n- If File Creation Test ✅ and CLI Test ❌:")
print("  → Fix JSON format structure")
print("\n- If File Creation Test ❌:")
print("  → Fix prepare_for_submission method")
print("\n- If CalcJob creates wrong file:")
print("  → Module caching issue, restart kernel")
print("\n- If file exists but CLI fails:")
print("  → Working directory or CLI argument issue")

print("\n" + "="*50)
print("🏁 DIAGNOSTIC COMPLETE")
print("Use the results above to identify and fix the specific issue.")
print("="*50)

## 8. Quick Fix Test

Once you've identified the issue, use this cell to test potential fixes quickly.

In [ ]:
# Quick fix testing area
print("=== QUICK FIX TEST ===")

# This cell can be used to test specific fixes based on the diagnosis above
# For example:

# Test 1: Try different JSON structure
# Test 2: Test with modified prepare_for_submission 
# Test 3: Test CLI with different arguments

print("Use this cell to test specific fixes based on the diagnosis above.")
print("\nCommon fixes to try:")
print("1. Restart kernel and re-run if module caching issue")
print("2. Modify JSON structure if CLI format issue")
print("3. Fix prepare_for_submission if file creation issue")
print("4. Adjust CLI arguments if interface issue")